# Modules

In [1]:
import re #module pour les expressions régulières
import os #module pour la gestion de répertoires/fichiers
import time, datetime #pour générer des temps

# Fonctions

### Récupération d'infos

In [2]:
# RECUPERER LES FICHIERS A TRAITER
def get_files(rep):
    """
    Entrée : nom du repertoire avec les fichiers txt
    Sortie : liste des fichiers à traiter
    """
    
    liste_fichiers = [os.path.join(rep,f) for f in os.listdir(rep)]
    
    return liste_fichiers

# RECUPERER LES ADVERBIAUX (unité et étiquette) ET LEURS INDICES DE DEBUT ET DE FIN
# (les indices par rapport à un fichier sans balise et sans retours à la ligne)
def get_indices_adverbiaux(doc,regEx_balise):
    """
    Entrée : STR(contenu textuel du fichier résultat Unitex)
    Sortie : dico {TUP(INT(indice de début),INT(indice de fin)):STR(type de l'adverbial)}
    """
    
    dico_indices_type = dict() # {(INT(indice_début),INT(indice_fin)):STR(type d'adverbial)}
    # dico_indices_adverbial = dict() # {(INT(indice_début),INT(indice_fin)):STR(adverbial)}

    soustraire_pour_debut = 0 #pour la correspondance des indices dans le contenu textuel SANS balises

    for m in re.finditer(regEx_balise,doc): # pour toutes les correspondances
        type_balise = m.group(2) # récupération du type d'adverbial
        adverbial = m.group(3) # récupération de l'adverbial
    
        # Valeurs nécessaires pour retrouver l'indice dans le contenu SANS balises
        longueur_balise_ouvrante = len(m.group(1))
        longueur_adverbial = len(adverbial)
        longueur_balise_fermante = len(m.group(4))
    
        soustraire_pour_debut += longueur_balise_ouvrante # pour l'adverbial courant
        
        # Calcul des indices de début et de fin
        debut = m.start(3)-soustraire_pour_debut # indice de début
        fin = debut+longueur_adverbial # indice de fin
    
        soustraire_pour_debut += longueur_balise_fermante # pour les adverbiaux suivants
    
        # Remplissage des dicos
        # dico_indices_adverbial[(debut,fin)]=adverbial
        dico_indices_type[(debut,fin)]=type_balise
        
    return dico_indices_type #, dico_indices_adverbial

# SUPPRIMER LES BALISES
def supprime_balises(m):
    """
    Entrée : objet match
    (résultat de correspondance avec la regEx "(<([^>]+)>)([^<]+)(</[^>]+>)")
    Sortie : la chaîne de caractère qui était entre les balises
    """
    return m.group(3)

# RECUPERER LES INDICES DE DEBUT ET DE FIN DES PARAGRAPHES
# (par rapport au contenu textuel sans balises, sans retours à la ligne)
def get_indices_para(paragraphes):
    """
    Trouve la position des paragraphes (indices de début et de fin dans le fichier .ac).
    Entrée : liste de paragraphes ["paragraphe 1", "paragraphe 2"]
    Sortie : dico {TUP(INT(indice de début),INT(indice de fin)):STR(paragraphe)}
    """

    # Les indices de début et de fin sont ceux dans un fichier .ac
    # càd où tous les retours à la ligne sont remplacés par un espace
    # et où il n'y a plus de balises Unitex
    dico_indices_paragraphe = dict()
    longueur_cumulative = 0 #somme itérative des longueurs des paragraphes, pour calculer les indices des caractères
    for para in paragraphes : #pour chaque paragraphe
        longueur_para = len(para) #on stocke sa longueur
        debut = longueur_cumulative
        fin = longueur_cumulative + longueur_para

        dico_indices_paragraphe[(debut,fin)] = para #on stocke les infos

        longueur_cumulative += longueur_para +1 #incrémentation de la longueur cumulative

    return dico_indices_paragraphe

### Elaboration des fichiers Glozz

In [3]:
def replace_nom_fichier(m):
    """
    Permet de récupérer le nom du fichier pour générer les fichiers .ac et .aa.
    """
    
    return m.group(1)

### FICHIERS .AC ###
def write_ac(rep_ac,paragraphes, nom = "sortie"):
    """
    Entrée :
    - répertoire de sortie
    - liste des paragraphes du fichier [STR(paragraphe 1),STR(paragraphe 2)]
    - éventuellement, nom du fichier ("sortie" par défaut)
    Sortie :
    - aucune
    """

    try: #on essaye de créer le répetoire de sortie
        os.mkdir(rep_ac)
    except OSError: #si ce n'est pas possible, c'est qu'il existe déjà
        pass

    with open(os.path.join(rep_ac,nom+".ac"),'w',encoding = "utf-8") as out_ac:
        out_ac.write(" ".join(paragraphes))
        
### FICHIERS .AA ###
# OBTENIR L'IDENTIFIANT POUR LE FICHIER .AA
def build_corpusHashcode(nom,rep_ac):
    """
    Génère un identifiant pour lier un fichier .aa à un fichier .ac.
    Entrée :
    - le nom du fichier (même nom pour le fichier .aa et .ac)
    - le répertoire où se trouve le fichier .ac à associer au fichier .aa
    Sortie : l'identifiant qui permettra d'associer le fichier .aa au fichier .ac (chaîne de caractères)
    """

    path_f_ac = os.path.join(rep_ac,nom+".ac") #reconstitution du chemin du fichier .ac à associer au fichier .aa
    f_ac = open(path_f_ac,'rb').read() #ouverture et lecture du fichier en flux d'octets
    length = len(f_ac) #taille du fichier ouvert

    hashcode = str(length)+"-" #première partie de l'identifiant

    code = 1 #va accueillir la deuxième partie de l'identifiant
    for i in range(0,length):
        n = int(f_ac[i]) #lecture octet par octet
        if n != 0 : #si l'octet ne vaut pas 0
            code *= n #on multiplie le code par la valeur de l'octet
            code = code%99999999 #et on fait le modulo

    hashcode += str(code) #ajout de la 2e partie de l'identifiant
    
    return hashcode

# OBTENIR UN TEMPS
# (nécessaire pour les identifiants d'unités)
def time_ms():
    """
    Donne la date sous la forme du temps, en millisecondes (nb entier), écoulé depuis le 1/1/1970
    """

    # time.time() donne le temps écoulé depuis le 1/1/1970 en secondes
    # on multiplie par 1000 pour avoir cette durée en millisecondes
    # on transforme le résultat en entier
    return int(time.time() * 1000)

# ASSOCIER UN TEMPS DE CREATION AUX PARAGRAPHES
# (nécessaire pour les identifiants d'unités)
def get_times(dico_indices_paragraphe):
    """
    Entrée : dico {TUP(INT(indice de début),INT(indice de fin)):STR(paragraphe)}
    Sortie : dico {TUP(INT(indice de début),INT(indice de fin)):INT(temps de création)}
    """

    dico_indicesPara_temps = dict()
    
    temps = time_ms() #temps initial, celui pour le 1er paragraphe
    for couple in dico_indices_paragraphe: # pour chaque paragraphe
        dico_indicesPara_temps[couple] = temps # on stocke le temps
        temps += 1 # on augmente de 1

    return dico_indicesPara_temps

# CONSTRUIRE LES UNITES CORRESPONDANT AUX PARAGRAPHES
def build_unit_para(id_annotateur,debut_fin,temps_creation):
    """
    Construit l'unité XML correspondant à un paragraphe pour le fichier .aa.
    Entrée :
    - l'identifiant d'annotation (ex. : "aetienne")
    - le tuple d'indices TUP(INT(debut de l'unité), INT(fin de l'unité))
    - le temps de creation de l'unité INT(temps de création)
    """

    t = str(temps_creation) # conversion du temps en chaîne de caractères
    
    #Construction de l'unité
    unit = '<unit id="'+id_annotateur+"_"+t+'">\n\t<metadata>\n\t\t<author>'+id_annotateur+'</author>\n\t\t<creation-date>'+t+'</creation-date>\n\t\t<lastModifier>n/a</lastModifier>\n\t\t<lastModificationDate>0</lastModificationDate>\n\t</metadata>\n\t<characterisation>\n\t\t<type>paragraph</type>\n\t\t<featureSet/>\n\t</characterisation>\n\t<positioning>\n\t\t<start>\n\t\t\t<singlePosition index="'+str(debut_fin[0])+'"/>\n\t\t</start>\n\t\t<end>\n\t\t\t<singlePosition index="'+str(debut_fin[1])+'"/>\n\t\t</end>\n\t</positioning>\n</unit>'
    
    return unit

# ASSOCIER UN TEMPS DE CREATION AUX UNITES D'ADVERBIAUX
def get_adverbial_time(dico_indicesPara_temps,dico_indices_type):
    """
    Donne un temps de création pour chaque unité de terme d'émotion (nécessaire pour le fichier .aa).
    Entrée :
    - dico, pour les paragraphes, {TUP(INT(indice de début),INT(indice de fin)) : INT(temps de création)}
    - dico, pour les adverbiaux, {TUP(INT(indice de début),INT(indice de fin)) : STR(type d'adverbial)}
    Sortie :
    - dico, pour les adverbiaux, {TUP(INT(indice de début),INT(indice de fin)) : INT(temps de création)}
    """

    temps = max(dico_indicesPara_temps.values()) #on part du temps du dernier paragraphe
    
    dico_indicesAdverbiaux_temps = dict()
    for couple in dico_indices_type: # pour chaque adverbial
        temps += 1 # augmentation du temps
        dico_indicesAdverbiaux_temps[couple] = temps # stockage de l'info

    return dico_indicesAdverbiaux_temps

# CONSTRUIRE LES UNITES CORRESPONDANT AUX ADVERBIAUX
def build_unit_adverbial(id_annotateur,debut_fin,temps_creation,type_adverbial):
    """
    Entrée :
    - identifiant de l'annotateur (ex. "aetienne")
    - tuple d'indices de début et de fin pour l'adverbial TUP(INT(début),INT(fin))
    - temps de création associée au tuple INT(temps de création)
    - type d'adverbial associé au tuple STR(type d'adverbial)
    Sortie :
    - l'unité XML à mettre dans le fichier .aa (chaîne de caractères)
    """
    
    t = str(temps_creation) # transformation du temps en chaîne de caractères

    #Construction de l'unité
    unit = '<unit id="'+id_annotateur+"_"+t+'">\n\t<metadata>\n\t\t<author>'+id_annotateur+'</author>\n\t\t<creation-date>'+t+'</creation-date>\n\t\t<lastModifier>n/a</lastModifier>\n\t\t<lastModificationDate>0</lastModificationDate>\n\t</metadata>\n\t<characterisation>\n\t\t<type>'+type_adverbial+'</type>\n\t\t<featureSet></featureSet>\n\t</characterisation>\n\t<positioning>\n\t\t<start>\n\t\t\t<singlePosition index="'+str(debut_fin[0])+'"/>\n\t\t</start>\n\t\t<end>\n\t\t\t<singlePosition index="'+str(debut_fin[1])+'"/>\n\t\t</end>\n\t</positioning>\n</unit>'

    return unit

# ECRIRE LE FICHIER .AA
def unitex2aa(
    rep_aa,rep_ac,id_annotateur,
    dico_indicesPara_temps,
    dico_indices_type,dico_indicesAdverbiaux_temps,
    nom="sortie"
):
    """
    Génère le fichier .aa.
    Entrée :
    - chemin du répertoire avec le fichier .aa
    - chemin du répertoire avec le fichier .ac
    - identifiant de l'annotateur (ex. "aetienne")
    - dico, pour les paragraphes, {TUP(INT(indice de début),INT(indice de fin)) : INT(temps de création)}
    - dico, pour les adverbiaux, {TUP(INT(indice de début),INT(indice de fin)) : STR(type d'adverbial)}
    - dico, pour les adverbiaux, {TUP(INT(indice de début),INT(indice de fin)) : INT(temps de création)}
    - facultatif : nom du fichier
    Sortie : rien

    """

    debut_aa = '<?xml version="1.0" encoding="UTF-8" standalone="no"?>\n<annotations>\n<metadata corpusHashcode="'
    corpusHashcode=build_corpusHashcode(nom,rep_ac)
    debut_aa_suite = '"/>\n'
    fin_aa = '\n</annotations>'

        
    # pourquoi ne pas faire un dictionnaire directement avec les indices des paragraphes et les temps de création ?
    unites_para = [
        build_unit_para(id_annotateur,couple,dico_indicesPara_temps[couple])
        for couple in dico_indicesPara_temps
    ]
    
    unites_adverbiaux = [
        build_unit_adverbial(id_annotateur,couple,dico_indicesAdverbiaux_temps[couple],dico_indices_type[couple])
        for couple in dico_indices_type
    ]

    try: #on essaye de créer le répetoire de sortie
        os.mkdir(rep_aa)
    except OSError: #si ce n'est pas possible, c'est qu'il existe déjà
        pass

    with open(os.path.join(rep_aa,nom+".aa"),'w',encoding = "utf-8") as out_aa:
        out_aa.write(debut_aa+corpusHashcode+debut_aa_suite)
        out_aa.write("\n".join(unites_para)+"\n")
        out_aa.write("\n".join(unites_adverbiaux))
        out_aa.write(fin_aa)

# Exécution

#### Données

In [4]:
### CHEMINS ###
# Chemin du répertoire avec les fichiers à convertir
rep_txt = "./GlozzFiles/txt"

# Chemin du répertoire où créer le fichier .ac
rep_ac = "./GlozzFiles/ac"

# Chemin du répertoire où créer le fichier .aa
rep_aa = "./GlozzFiles/aa"

# Identifiant de l'annotateur
# (pour les fichiers .aa)
################## A CHANGER ##########################################
id_annotateur = "aetienne"
#######################################################################

### EXPRESSIONS REGULIERES ###
regEx_paragraphes = re.compile(r"\r?\n+") # retrouver les paragraphes
regEx_balise = re.compile(r"(<([^>]+)>)([^<]+)(</[^>]+>)") # retrouver les adverbiaux repérés par Unitex

#### Récupération d'infos

In [12]:
liste_fichiers_txt = get_files(rep_txt) # récupération des fichiers à traiter

for txt_file in liste_fichiers_txt :
    print(txt_file)
    
    #---Etape 1 : récupération des informations---#
    
    # Récupération du contenu du fichier résultat d'Unitex
    doc = open(txt_file,'r').read()

    # Léger nettoyage
    doc = doc.replace("{S}","")
    doc = doc.replace("_","-")

    # Suppression des balises
    doc_sans_balises = re.sub(regEx_balise,supprime_balises,doc)

    # Récupération des paragraphes
    paragraphes = regEx_paragraphes.split(doc_sans_balises)

    # Récupération des indices de début et de fin des paragraphes
    dico_indices_paragraphe = get_indices_para(paragraphes)

    # Récupération des adverbiaux
    # (indices de début et de fin + type d'adverbial)
    dico_indices_type = get_indices_adverbiaux(doc,regEx_balise)

    # Temps de création des unités de paragraphes
    dico_indicesPara_temps = get_times(dico_indices_paragraphe)

    # Temps de création des unités d'adverbiaux
    dico_indicesAdverbiaux_temps = get_adverbial_time(dico_indicesPara_temps,dico_indices_type)
    
    #---Etape 2 : écriture des fichiers de sortie---#
    # Ecriture du fichier .ac
    nom_ac_aa = re.sub(".+/([^.]+)\.txt",replace_nom_fichier,txt_file)
    #print(nom_ac)
    write_ac(rep_ac, paragraphes, nom_ac_aa)

    # Ecriture du fichier .aa
    unitex2aa(
        rep_aa,rep_ac,id_annotateur,
        dico_indicesPara_temps,
        dico_indices_type,dico_indicesAdverbiaux_temps,
        nom_ac_aa
    )
    

./GlozzFiles/txt/chap10_resultat.txt
./GlozzFiles/txt/chap11_resultat.txt
./GlozzFiles/txt/chap12_resultat.txt
./GlozzFiles/txt/chap13_resultat.txt
./GlozzFiles/txt/chap14_resultat.txt
./GlozzFiles/txt/chap15_resultat.txt
./GlozzFiles/txt/chap6_resultat.txt
./GlozzFiles/txt/chap7_resultat.txt
./GlozzFiles/txt/chap8_resultat.txt
./GlozzFiles/txt/chap9_resultat.txt
